In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics

import models.resnet as resnet

import sys

import data_augmentation as da
import data_visualisation as dv

2024-08-09 16:07:54.557159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 16:07:55.085099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-08-09 16:07:55.085159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-08-09 16:07:55.085165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

# Load Data

In [2]:
# load data
thismodule = sys.modules[__name__]

with np.load('data/DAT_China_pretrain_1s_HB.npz', allow_pickle=True) as data:
    for k in data.keys():
        print(k)
        if 'text' in k:
            setattr(thismodule, k, data[k])
        else:
            setattr(thismodule, k, data[k].astype(float))

X
X_folds
Y
Y_text


In [3]:
print("X", X.shape)
print("X_folds", X_folds.shape)
print("Y", Y.shape)
print("Y_text", Y_text.shape)

X (282615, 100, 12)
X_folds (282615,)
Y (282615,)
Y_text (282615,)


In [4]:
# take fold 7 as test set
X_train = X[X_folds != 7]
Y_train = Y[X_folds != 7]
X_test = X[X_folds == 7]
Y_test = Y[X_folds == 7]

print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

X_train (247380, 100, 12)
Y_train (247380,)
X_test (35235, 100, 12)
Y_test (35235,)


In [5]:
def augment_ecg_signal(signal):
    drifted_signal, _ = da.add_random_baseline_drift(signal, strength_range=(1.5,2.5), drift_wavelength_range=(300,500))
    noised_drifted_signal = da.add_random_noise(drifted_signal, (0, 0.2))
    return noised_drifted_signal

def augment_ecg_signal_batch(signals, labels, batch_size):
    while True:
        
        indices = np.random.randint(0, signals.shape[0], size=batch_size)
        batch = signals[indices]
        batch_labels = labels[indices]
        
        augmented_batch = np.array([augment_ecg_signal(sample) for sample in batch])
        
        yield (augmented_batch, batch_labels)

# Create model

In [6]:
num_classes = 1
X_input = keras.Input(shape=X.shape[1:])

out, _ = resnet.model(X_input, num_classes=num_classes, filters = [16, 16], kernels = [5, 3], layers=10, hidden_units=128)
model = keras.Model(inputs=X_input, outputs=out)

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', metrics.binary_accuracy])
model.summary()

2024-08-09 16:00:36.413615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-09 16:00:36.471776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-09 16:00:36.471973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-09 16:00:36.472427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 12)]    0           []                               
                                                                                                  
 conv1 (Conv1D)                 (None, 96, 16)       976         ['input_1[0][0]']                
                                                                                                  
 bn1 (BatchNormalization)       (None, 96, 16)       64          ['conv1[0][0]']                  
                                                                                                  
 activation (Activation)        (None, 96, 16)       0           ['bn1[0][0]']                    
                                                                                              

# Train model

In [7]:
batch_size = 64

train_generator = augment_ecg_signal_batch(X_train, Y_train, batch_size)

history = model.fit(train_generator, epochs=4, batch_size=batch_size, validation_data=(X_test, Y_test), steps_per_epoch=X_train.shape[0]//batch_size, verbose=1)

Epoch 1/4


2024-08-09 16:00:44.401623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-08-09 16:00:45.310698: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-08-09 16:00:45.314773: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x2e336470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-09 16:00:45.314794: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-08-09 16:00:45.319236: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-09 16:00:45.400452: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of t

3865/3865 [==============================] - 92s 21ms/step - loss: 0.0332 - accuracy: 0.9902 - precision: 0.9928 - recall: 0.9853 - binary_accuracy: 0.9902 - val_loss: 0.0301 - val_accuracy: 0.9927 - val_precision: 0.9995 - val_recall: 0.9842 - val_binary_accuracy: 0.9927
Epoch 2/4
3865/3865 [==============================] - 79s 20ms/step - loss: 0.0173 - accuracy: 0.9961 - precision: 0.9979 - recall: 0.9934 - binary_accuracy: 0.9961 - val_loss: 0.0265 - val_accuracy: 0.9944 - val_precision: 0.9975 - val_recall: 0.9900 - val_binary_accuracy: 0.9944
Epoch 3/4
3865/3865 [==============================] - 77s 20ms/step - loss: 0.0143 - accuracy: 0.9969 - precision: 0.9986 - recall: 0.9945 - binary_accuracy: 0.9969 - val_loss: 0.0273 - val_accuracy: 0.9952 - val_precision: 0.9997 - val_recall: 0.9897 - val_binary_accuracy: 0.9952
Epoch 4/4
3865/3865 [==============================] - 79s 20ms/step - loss: 0.0121 - accuracy: 0.9973 - precision: 0.9988 - recall: 0.9953 - binary_accuracy: 0.

In [9]:
# save model
model.save('model-weights/Norm_RBBB_resnet_classifier.h5')

In [3]:
X_input = keras.Input(shape=X.shape[1:])
_, feature_extractor = resnet.model(X_input, num_classes=1, filters = [16, 16], kernels = [5, 3], layers=10, hidden_units=128)
feature_extractor_model = keras.Model(inputs=X_input, outputs=feature_extractor)

# load weights from trained model
feature_extractor_model.load_weights('model-weights/Norm_RBBB_resnet_classifier.h5', by_name=True)

# save feature extractor model
feature_extractor_model.save('model-weights/Norm_RBBB_resnet_feature_extractor.h5')

2024-08-09 16:07:58.977973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-09 16:07:59.034863: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-09 16:07:59.035065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-09 16:07:59.035453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF